# State Charts in Python
## Nicer Code and Stuff with Sismic
### nicer runtime too!

## But first...
# Finite State Machines

<img src="statemachine_light2.svg" alt="state machine diagram">

# Harel State Charts
## Statecharts: A Visual Formalism for Complex Systems `87

<img src="statecharts_light.svg" style="height:80%"/>

# My light also has a fan
## With 4 levels: Off, low, med, hi

<img src="statemachine_fan.svg" style="width: 600px"/>

<img src="statecharts_fan.svg"/>

# How do we model both light and fan?
### States: {fan levels} x {light on, light off} = 8 only inside :(

<img src="statecharts_full.svg"/>

In [49]:
# Light State Machine
open("statemachine_light.yml", "wb").write("""
statechart:
  name: Finite State Machine
  root state: 
    name: LightSwitch
    initial: NoPower
    states:
      - name: NoPower
        transitions:
          - event: flick
            target: LightOn
      - name: LightOn
        transitions:
          - event: flick
            target: NoPower
          - event: remotePressedOff
            target: LightOff
      - name: LightOff
        transitions:
          - event: remotePressedOn
            target: LightOn
          - event: flick
            target: justTurnedOff
      - name: justTurnedOff
        transitions:
          - event: flick
            target: LightOff
          - target: NoPower
            guard: after(2)
""")

!dot statemachine_light.dot -T svg > statemachine_light.svg

In [99]:
# Light State Chart
from sismic.io import import_from_yaml, export_to_plantuml

statechart = import_from_yaml("""
statechart:
  name: State Chart
  root state:
    name: LightSwitch
    initial: NoPower
    states:
      - name: NoPower
        transitions:
          - event: flick
            target: HasMemory
      - name: HasMemory
        initial: WithPower
        states:
          - name: WithPower
            initial: LightOn
            states:
              - name: history state
                type: shallow history
              - name: LightOn
                transitions:
                  - event: remotePressedOff
                    target: LightOff
              - name: LightOff
                transitions:
                  - event: remotePressedON
                    target: LightOn
            transitions:
              - event: flick
                target: JustTurnedOff
          - name: JustTurnedOff
            transitions:
              - event: flick
                target: history state
              - target: NoPower
                guard: after(2)
""")

export_to_plantuml(statechart, "statecharts_flick.puml");

In [98]:
# Fan State Chart
from sismic.io import import_from_yaml, export_to_plantuml

statechart = import_from_yaml("""
statechart:
  name: State Chart
  root state:
    name: FanSwitch
    initial: NoPower
    states:
      - name: NoPower
        transitions:
          - event: flick
            target: HasMemory
      - name: HasMemory
        initial: WithPower
        states:
          - name: WithPower
            initial: FanOff
            states:
              - name: history state
                type: shallow history
              - name: FanOff
              - name: FanLow
              - name: FanMed
              - name: FanHi
            transitions:
              - event: remotePressedOff
                target: FanOff
              - event: remotePressedLow
                target: FanLow
              - event: remotePressedMed
                target: FanMed
              - event: remotePressedHi
                target: FanHi
              - event: flick
                target: JustTurnedOff
          - name: JustTurnedOff
            transitions:
              - event: flick
                target: history state
              - target: NoPower
                guard: after(2)
""")

export_to_plantuml(statechart, "statecharts_fan.puml");

In [36]:
# Fan State Machine

open("statemachine_fan.yml", "wb").write("""
statechart:
  name: Finite State Machine
  root state: 
    name: FanSwitch
    initial: NoPower
    states:
      - name: NoPower
        transitions:
          - event: flick
            target: FanOff
      - name: FanOff
        transitions:
          - event: flick
            target: NoPower
          - event: remotePressedLow
            target: FanLow
          - event: remotePressedMed
            target: FanMed
          - event: remotePressedHi
            target: FanHi
      - name: FanLow
        transitions:
          - event: flick
            target: JustTurnedOffFromLow
          - event: remotePressedOff
            target: FanOff
          - event: remotePressedMed
            target: FanMed
          - event: remotePressedHi
            target: FanHi
      - name: FanMed
        transitions:
          - event: flick
            target: JustTurnedOffFromMed
          - event: remotePressedOff
            target: FanOff
          - event: remotePressedLow
            target: FanLow
          - event: remotePressedHi
            target: FanHi
      - name: FanHi
        transitions:
          - event: flick
            target: JustTurnedOffFromHi
          - event: remotePressedOff
            target: FanOff
          - event: remotePressedLow
            target: FanLow
          - event: remotePressedMed
            target: FanMed
      - name: JustTurnedOffFromLow
        transitions:
          - event: flick
            target: FanLow
          - target: NoPower
            guard: after(2)
      - name: JustTurnedOffFromMed
        transitions:
          - event: flick
            target: FanMed
          - target: NoPower
            guard: after(2)
      - name: JustTurnedOffFromHi
        transitions:
          - event: flick
            target: FanHi
          - target: NoPower
            guard: after(2)
""")
!python2 -m sismic_viz -o statemachine_fan.dot -T dot statemachine_fan.yml
!dot statemachine_fan.dot -T svg > statemachine_fan.svg

In [1]:
# Full State Chart
from sismic.io import import_from_yaml, export_to_plantuml

yaml = """statechart:
  name: State Chart for Realz
  root state:
    name: LightAndFanSwitch
    initial: NoPower
    states:
      - name: NoPower
        transitions:
          - event: flick
            target: HasMemory
      - name: HasMemory
        initial: WithPower
        states:
          - name: WithPower
            transitions:
              - event: flick
                target: JustTurnedOff
            initial: Components
            states:
              - name: history state
                type: deep history
              - name: Components
                parallel states:
                  - name: Light
                    initial: LightOn
                    states:
                      - name: LightOn
                        transitions:
                          - event: remotePressedLightOff
                            target: LightOff
                      - name: LightOff
                        transitions:
                          - event: remotePressedLightON
                            target: LightOn
                  - name: Fan
                    initial: FanOff
                    states:
                      - name: FanOff
                      - name: FanLow
                      - name: FanMed
                      - name: FanHi
                    transitions:
                      - event: remotePressedFanOff
                        target: FanOff
                      - event: remotePressedLow
                        target: FanLow
                      - event: remotePressedMed
                        target: FanMed
                      - event: remotePressedHi
                        target: FanHi
          - name: JustTurnedOff
            transitions:
              - event: flick
                target: history state
              - target: NoPower
                guard: after(2)
"""

open("statecharts_full.yaml", "wb").write(yaml)
statechart = import_from_yaml(yaml)
export_to_plantuml(statechart, "statecharts_full.puml");

## Running State Chart with Sismic

In [2]:
from sismic.interpreter import Interpreter

interpreter = Interpreter(statechart)
interpreter.execute()  # Enter states
print "start:", interpreter.configuration
print

(
    interpreter
    .queue("flick")
    .queue("remotePressedLightOff")
    .queue("remotePressedMed")
    .queue("flick")
    .execute()
)

interpreter.clock.time += 1
interpreter.queue("flick").execute()
print "end:", interpreter.configuration

start: ['LightAndFanSwitch', 'NoPower']

end: ['LightAndFanSwitch', 'HasMemory', 'WithPower', 'Components', 'Fan', 'Light', 'FanMed', 'LightOff']


## Exploring a State Chart Offline

In [108]:
# pip install git+https://github.com/drorspei/sismic_viz
!python2 -m sismic_viz -i statecharts_full.yaml

 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [22/May/2019 02:07:58] "GET / HTTP/1.1" 200 -
[25677:25696:0522/020758.897440:ERROR:browser_process_sub_thread.cc(217)] Waited 6 ms for network service
127.0.0.1 - - [22/May/2019 02:07:58] "GET /statechart.svg?1558483678.89 HTTP/1.1" 200 -
Opening in existing browser session.
127.0.0.1 - - [22/May/2019 02:08:01] "GET /?event=flick HTTP/1.1" 200 -
127.0.0.1 - - [22/May/2019 02:08:01] "GET /statechart.svg?1558483681.94 HTTP/1.1" 200 -
127.0.0.1 - - [22/May/2019 02:08:06] "GET /?event=remotePressedLightOff HTTP/1.1" 200 -
127.0.0.1 - - [22/May/2019 02:08:06] "GET /statechart.svg?1558483686.11 HTTP/1.1" 200 -
127.0.0.1 - - [22/May/2019 02:08:08] "GET /?event=remotePressedHi HTTP/1.1" 200 -
127.0.0.1 - - [22/May/2019 02:08:08] "GET /statechart.svg?1558483688.15 HTTP/1.1" 200 -
127.0.0.1 - - [22/May/2019 02:08:13] "GET /?event=flick HTTP/1.1" 200 -
127.0.0.1 - - [22/May/2019 02:08:13] "GET /statechart.svg?1558483693.08 H

In [3]:
def send_random_events(interpreter):
    from threading import Event, Thread
    import random
    event = Event()
    def inner():
        while not event.wait(1):
            rand_event = random.choice(list(set(filter(
                None,
                (trans.event for trans in interpreter.statechart.transitions)
            ))))
            interpreter.clock.time += 0.5
            interpreter.queue(rand_event).execute()
    Thread(target=inner).start()
    return event

## Viewing a State Chart During Runtime

In [4]:
from sismic_viz import server_to_bind

server = server_to_bind(statechart)
with server as callback:
    interpreter = Interpreter(statechart)
    interpreter.attach(callback)
    interpreter.execute()
    
    stop_event = send_random_events(interpreter)
    
    (lambda: raw_input("click enter to finish"))()
    stop_event.set()

 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [22/May/2019 02:28:24] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [22/May/2019 02:28:24] "GET /statechart.svg?1558484904.03 HTTP/1.1" 200 -
127.0.0.1 - - [22/May/2019 02:28:25] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [22/May/2019 02:28:25] "GET /statechart.svg?1558484905.24 HTTP/1.1" 200 -
127.0.0.1 - - [22/May/2019 02:28:26] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [22/May/2019 02:28:26] "GET /statechart.svg?1558484906.45 HTTP/1.1" 200 -
127.0.0.1 - - [22/May/2019 02:28:27] "GET /shutdown HTTP/1.1" 200 -


click enter to finish 
